In [ ]:
import cupy as np
from torchvision import datasets, transforms

dtype = np.float32

In [ ]:
training_data_set = datasets.MNIST(
    root        = "./data/MNIST/",
    train       = True,
    transform   = transforms.ToTensor,
    download    = True
    ).data.numpy() / 255

testing_data_set = datasets.MNIST(
    root        = "./data/MNIST/",
    train       = False,
    transform   = transforms.ToTensor,
    download    = True
    ).data.numpy() / 255

training_data_set = np.asarray(training_data_set, dtype)
testing_data_set  = np.asarray(testing_data_set , dtype)

training_data_set[training_data_set < 0.3] = 0
training_data_set[training_data_set >= 0.3] = 1
testing_data_set[testing_data_set < 0.3] = 0
testing_data_set[testing_data_set >= 0.3] = 1

In [ ]:
def random_separate(
        data_set_size   : int,
        separate_size   : int | tuple,
        seed            : int | None = None
        ):
    
    rs = np.random.RandomState(seed)
    if hasattr(separate_size, "__iter__"):
        _randIdx = rs.choice(data_set_size, sum(list(separate_size)), replace = False)
        randIdx = []
        for i in range(len(separate_size)):
            randIdx.append(_randIdx[sum(separate_size[: i]) : sum(separate_size[: i + 1])])
    else:
        randIdx = rs.choice(data_set_size, separate_size, replace = False)

    return randIdx


n_validation = 10000
n_training   = 60000 - n_validation
n_testing    = 10000

idx_training_data_set, idx_validation_data_set = random_separate(
    data_set_size = training_data_set.shape[0], 
    separate_size = (n_training, n_validation),
    seed          = 12345
    )

idx_testing_data_set = random_separate(
    data_set_size = testing_data_set.shape[0], 
    separate_size = n_testing,
    seed          = 12345
)

validation_data_set = training_data_set[idx_validation_data_set]
training_data_set   = training_data_set[idx_training_data_set]
testing_data_set    = testing_data_set[idx_testing_data_set]

#training_data_set   = training_data_set[idx_training_data_set]
#validation_data_set = training_data_set
#testing_data_set    = training_data_set

training_data_set.shape, validation_data_set.shape, testing_data_set.shape

In [ ]:
from Restricted_Boltzmann_Machine.Restricted_Boltzmann_Machine import RBM
from Restricted_Boltzmann_Machine.Training import RBM_training

rbm = RBM(
    visible_size  = 28 * 28,
    hidden_size   = 64,
    seed          = 1234,
    dtype         = np.float32
    )

training = RBM_training(
    rbm                 = rbm,
    epochs              = 40,
    training_samples    = training_data_set,
    validation_samples  = validation_data_set,
    testing_samples     = testing_data_set,
    batch_size          = 400,
    learning_rate       = 1e-4
)

training.start_training(1)


In [ ]:
n_figs = min(100, testing_data_set.shape[0])
testing = testing_data_set[:n_figs].reshape((n_figs, 28*28))
reconstruct, _, _ = rbm.forward(testing)

testing = testing.reshape((n_figs, 28, 28)) * 255
testing = testing.astype(np.int8)
reconstruct = reconstruct.reshape((n_figs, 28, 28)) * 255
reconstruct = reconstruct.astype(np.int8)


In [ ]:
from PIL import Image
import os

if os.path.exists("./figs/") is not True:
    os.mkdir("./figs/")

for i in range(n_figs):
    im_test = Image.fromarray(testing[i].get(), mode = "L")
    im_test.save(f"./figs/{i+1}_original.jpg")
    im_reconstruct = Image.fromarray(reconstruct[i].get(), mode = "L")
    im_reconstruct.save(f"./figs/{i+1}_reconstruct.jpg")
